In [25]:
import time
import sys
import numpy as np
import os
import argparse
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import gzip
import onnx
import tf2onnx
import onnxruntime as ort # Для проверки ONNX
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# --- Класс загрузки данных (ваш класс) ---
class MNISTDataLoader:
    def __init__(self, data_dir, one_hot=True, class_num=10):
        self.data_dir = data_dir
        self.one_hot = one_hot
        self.class_num = class_num

        # Проверяем существование директории
        if not os.path.isdir(self.data_dir):
            raise FileNotFoundError(f"Директория данных не найдена: {self.data_dir}")

        print(f"Загрузка данных из: {self.data_dir}")
        self.train_images, self.train_labels = self._load_data('train')
        self.test_images, self.test_labels = self._load_data('t10k')
        print("Данные загружены.")
        print(f"  Train: {self.train_images.shape}, {self.train_labels.shape}")
        print(f"  Test:  {self.test_images.shape}, {self.test_labels.shape}")

        # Добавляем свойство для количества тренировочных примеров
        self.num_train_examples = len(self.train_images)

    def _load_data(self, prefix):
        """Load MNIST data from .gz files"""
        image_path = os.path.join(self.data_dir, f'{prefix}-images-idx3-ubyte.gz')
        label_path = os.path.join(self.data_dir, f'{prefix}-labels-idx1-ubyte.gz')

        if not os.path.exists(image_path) or not os.path.exists(label_path):
             raise FileNotFoundError(f"Файлы данных MNIST не найдены в {self.data_dir} с префиксом '{prefix}'")

        try:
            # Load images
            with gzip.open(image_path, 'rb') as f:
                # Смещение 16 байт для заголовка IDX3
                images = np.frombuffer(f.read(), np.uint8, offset=16)
                # MNIST изображения 28x28 = 784
                images = images.reshape(-1, 784).astype(np.float32) / 255.0
                # images = images.reshape(-1, 28, 28, 1).astype(np.float32) / 255.0 # <--- ИЗМЕНЕНО ЗДЕСЬ


            # Load labels
            with gzip.open(label_path, 'rb') as f:
                # Смещение 8 байт для заголовка IDX1
                labels = np.frombuffer(f.read(), np.uint8, offset=8)

            # Convert to one-hot if needed
            if self.one_hot:
                # Используем tf.keras.utils.to_categorical для надежности
                one_hot_labels = tf.keras.utils.to_categorical(labels, num_classes=self.class_num)
                # labels = tf.one_hot(labels, self.class_num).numpy() # Альтернатива
                return images, one_hot_labels

            return images, labels
        except Exception as e:
            print(f"Ошибка при загрузке или обработке файлов {prefix}: {e}")
            raise

# --- Словари классов (как в оригинале) ---
dict_2class = {0:'Benign',1:'Malware'}
dict_10class_benign = {0:'BitTorrent',1:'Facetime',2:'FTP',3:'Gmail',4:'MySQL',5:'Outlook',6:'Skype',7:'SMB',8:'Weibo',9:'WorldOfWarcraft'}
dict_10class_malware = {0:'Cridex',1:'Geodo',2:'Htbot',3:'Miuref',4:'Neris',5:'Nsis-ay',6:'Shifu',7:'Tinba',8:'Virut',9:'Zeus'}
dict_20class = {0:'BitTorrent', 1:'Facetime', 2:'FTP', 3:'Gmail', 4:'MySQL',
               5:'Outlook', 6:'Skype', 7:'SMB', 8:'Weibo', 9:'WorldOfWarcraft',
               10:'Cridex', 11:'Geodo', 12:'Htbot', 13:'Miuref', 14:'Neris',
               15:'Nsis-ay', 16:'Shifu', 17:'Tinba', 18:'Virut', 19:'Zeus'}

# --- Создание модели (максимально близко к оригиналу) ---
def create_original_model(class_num, include_softmax=True):
    """
    Создает модель CNN с использованием Keras Functional API,
    ожидающую вход (28, 28, 1).

    Args:
        class_num (int): Количество выходных классов.
        include_softmax (bool): Включать ли Softmax.

    Returns:
        tf.keras.Model: Скомпилированная модель Keras Functional API.
    """
    # Определяем входной слой
    model = models.Sequential(name=f"SequentialOriginalLike_{class_num}class")
    # НЕ добавляем Input слой явно
    # Вход плоский (784), Reshape внутри
    model.add(layers.Reshape((28, 28, 1), input_shape=(784,))) # <--- ВАЖНО

    # Сверточные слои
    model.add(layers.Conv2D(32, (5, 5), padding='same', activation='relu', name='conv1'))
    # Используем padding='valid' по умолчанию для MaxPooling, как было в рабочем коде неявно
    model.add(layers.MaxPooling2D((2, 2), name='pool1')) # <--- padding='valid' по умолч.

    model.add(layers.Conv2D(64, (5, 5), padding='same', activation='relu', name='conv2'))
    model.add(layers.MaxPooling2D((2, 2), name='pool2')) # <--- padding='valid' по умолч.

    # Полносвязные слои
    model.add(layers.Flatten(name='flatten'))
    model.add(layers.Dense(1024, activation='relu', name='dense1'))
    model.add(layers.Dropout(0.5, name='dropout'))

    # Выходной слой
    if include_softmax:
        model.add(layers.Dense(class_num, activation='softmax', name='output_softmax'))
        loss_function = tf.keras.losses.CategoricalCrossentropy()
    else:
        model.add(layers.Dense(class_num, name='output_logits'))
        loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # Создаем модель Functional API
    # model = models.Model(inputs=input_tensor, outputs=output_tensor, name=f"OriginalCNN_Functional_{class_num}class")

    # Компиляция модели
    # optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])
    model.build(input_shape=(None, 784))

    print("--- Модель (Functional API) создана ---")
    model.summary()
    print("---------------------------------------")
    return model

# --- Обучение модели ---
def train_model(model, data_loader, train_steps, batch_size, model_save_path):
    """
    Обучает модель с использованием model.fit и сохраняет лучшую версию.

    Args:
        model (tf.keras.Model): Скомпилированная модель Keras.
        data_loader (MNISTDataLoader): Загрузчик данных.
        train_steps (int): Общее количество шагов обучения (как TRAIN_ROUND).
        batch_size (int): Размер батча.
        model_save_path (str): Путь для сохранения лучшей модели (.h5).

    Returns:
        tf.keras.callbacks.History: История обучения.
    """
    # Рассчитываем количество эпох
    steps_per_epoch = max(1, data_loader.num_train_examples // batch_size)
    epochs = max(1, train_steps // steps_per_epoch)
    print(f"Расчетное количество эпох: {epochs} ({train_steps} шагов / {steps_per_epoch} шагов в эпохе)")

    # Создаем директорию для сохранения, если ее нет
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

    # Колбэк для сохранения лучшей модели по val_accuracy
    checkpoint = ModelCheckpoint(
        filepath=model_save_path,
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=False, # Сохраняем всю модель
        mode='max',
        verbose=1
    )
    # Колбэк для ранней остановки, если улучшений нет
    early_stopping = EarlyStopping(
        monitor='val_accuracy',
        patience=10, # Количество эпох без улучшений перед остановкой (настройте)
        verbose=1,
        restore_best_weights=True # Восстановить лучшие веса в конце
    )

    print(f"Начало обучения на {epochs} эпох...")
    history = model.fit(
        data_loader.train_images,
        data_loader.train_labels,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(data_loader.test_images, data_loader.test_labels),
        callbacks=[checkpoint, early_stopping],
        verbose=1 # Показываем прогресс
    )
    print("Обучение завершено.")
    # Модель уже содержит лучшие веса благодаря restore_best_weights=True
    return history

# --- Оценка модели ---
def evaluate_model(model, data_loader, class_num, data_dir):
    """
    Оценивает модель на тестовых данных и выводит метрики.

    Args:
        model (tf.keras.Model): Обученная модель Keras.
        data_loader (MNISTDataLoader): Загрузчик данных.
        class_num (int): Количество классов.
        data_dir (str): Директория данных (для выбора словаря).
    """
    print("\nНачало оценки модели на тестовых данных...")
    y_true_one_hot = data_loader.test_labels
    y_true = np.argmax(y_true_one_hot, axis=1)

    # Получаем предсказания (логиты или вероятности, в зависимости от include_softmax)
    predictions = model.predict(data_loader.test_images)

    # Если модель выводит логиты, применяем argmax
    # Если выводит вероятности (softmax), argmax тоже сработает
    y_pred = np.argmax(predictions, axis=1)

    # Выбор словаря для имен классов
    folder = os.path.basename(data_dir)
    dict_labels = {}
    if class_num == 2: dict_labels = dict_2class
    elif class_num == 20: dict_labels = dict_20class
    elif class_num == 10:
        if folder.startswith('Benign'): dict_labels = dict_10class_benign
        elif folder.startswith('Malware'): dict_labels = dict_10class_malware
    label_names = [dict_labels.get(i, f'Class_{i}') for i in range(class_num)]

    # Расчет метрик
    accuracy = accuracy_score(y_true, y_pred)
    # average=None возвращает метрики для каждого класса
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average=None, labels=list(range(class_num)), zero_division=0
    )
    # Средние метрики (можно использовать 'macro' или 'weighted')
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )

    print("--- Результаты Оценки ---")
    print(f"Общая точность (Accuracy): {accuracy:.4f}")
    print(f"Precision (Macro Avg):    {precision_macro:.4f}")
    print(f"Recall (Macro Avg):       {recall_macro:.4f}")
    print(f"F1-Score (Macro Avg):     {f1_macro:.4f}")
    print("\nМетрики по классам:")
    print(f"{'Класс':<6} {'Имя':<18} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'Support':<8}")
    print("-" * 70)
    acc_list_str = []
    for i in range(class_num):
        name = label_names[i]
        print(f"{i:<6} {name:<18} {precision[i]:<10.4f} {recall[i]:<10.4f} {f1[i]:<10.4f} {support[i]:<8}")
        # Формируем строку для записи в файл (как в оригинале)
        acc_list_str.append([str(i), name, f"{precision[i]:.4f}", f"{recall[i]:.4f}"])

    # Запись в файл (добавляем в конец)
    try:
        with open('out_tf2.txt', 'a') as f:
            f.write("\n")
            t = time.strftime('%Y-%m-%d %X', time.localtime())
            f.write(t + "\n")
            f.write(f'DATA_DIR: {data_dir}\n')
            f.write(f'CLASS_NUM: {class_num}\n')
            f.write(f'MODEL: Original TF2 Reimplementation\n')
            f.write("Класс, Имя, Precision, Recall\n")
            for item in acc_list_str:
                f.write(', '.join(item) + "\n")
            f.write(f'Total accuracy: {accuracy:.4f}\n')
            f.write(f'Precision (Macro): {precision_macro:.4f}\n')
            f.write(f'Recall (Macro): {recall_macro:.4f}\n')
            f.write(f'F1-Score (Macro): {f1_macro:.4f}\n\n')
        print("\nРезультаты оценки записаны в out_tf2.txt")
    except Exception as e:
        print(f"\nОшибка записи результатов оценки в файл: {e}")

    print("-------------------------")

In [26]:
import traceback # Добавим для вывода стека ошибки

def export_to_onnx(model_to_convert, onnx_save_path):
    """
    Экспортирует модель Keras в формат ONNX, используя Functional API wrapper.

    Args:
        model_to_convert (tf.keras.Model): Обученная модель Keras (Sequential или Functional).
        onnx_save_path (str): Путь для сохранения ONNX модели.

    Returns:
        bool: True если экспорт успешен, иначе False.
    """
    print(f"\nЭкспорт модели в ONNX: {onnx_save_path}")
    try:
        # 1. Определяем входную сигнатуру (плоский вектор 784)
        # Имя 'input_flat' должно совпадать с именем входного слоя модели или нового Input
        input_signature = [tf.TensorSpec((None, 784), tf.float32, name='input_flat')]

        # --- Создаем Functional API Wrapper ---
        # Создаем явный входной слой с тем же shape и именем, что ожидает модель
        func_input = tf.keras.Input(
            shape=(784,), batch_size=None, dtype=tf.float32, name='input_flat'
        )
        # Пропускаем вход через существующую модель
        # model_to_convert здесь - это ваша обученная Sequential модель
        func_output = model_to_convert(func_input)
        # Создаем новую модель-обертку Functional API
        wrapper_model = tf.keras.Model(inputs=func_input, outputs=func_output, name=model_to_convert.name + "_wrapper")
        print("--- Структура модели-обертки (для ONNX) ---")
        wrapper_model.summary()
        print("-------------------------------------------")
        # --- Обертка создана ---

        # 2. Конвертируем модель-ОБЕРТКУ
        model_proto, external_tensor_storage = tf2onnx.convert.from_keras(
            wrapper_model, # <--- Конвертируем обертку!
            input_signature=input_signature,
            opset=13,
            output_path=onnx_save_path
        )
        print(f"Модель успешно экспортирована в ONNX: {onnx_save_path}")
        return True

    except Exception as e:
        print(f"Ошибка экспорта в ONNX: {e}")
        print("--- Traceback ---")
        print(traceback.format_exc()) # Выводим полный traceback для диагностики
        print("-----------------")
        return False
    


    # --- Проверка ONNX модели ---
def test_onnx_model(onnx_path, data_loader):
    """
    Загружает ONNX модель и выполняет инференс на нескольких примерах.

    Args:
        onnx_path (str): Путь к ONNX модели.
        data_loader (MNISTDataLoader): Загрузчик данных для тестовых примеров.
    """
    if not onnx_path or not os.path.exists(onnx_path):
        print("Пропуск проверки ONNX: файл не найден.")
        return

    print(f"\nПроверка ONNX модели: {onnx_path}")
    try:
        sess = ort.InferenceSession(onnx_path, providers=ort.get_available_providers()) # Используем доступные провайдеры
        input_name = sess.get_inputs()[0].name
        output_name = sess.get_outputs()[0].name
        print(f"  ONNX Input: '{input_name}', Output: '{output_name}'")

        # Берем несколько тестовых примеров
        sample_images = data_loader.test_images[:5]
        sample_labels_true = np.argmax(data_loader.test_labels[:5], axis=1)

        # Выполняем инференс
        outputs_onnx = sess.run([output_name], {input_name: sample_images.astype(np.float32)})[0]
        predictions_onnx = np.argmax(outputs_onnx, axis=1)

        print("  Примеры предсказаний ONNX:")
        for i in range(len(sample_images)):
            print(f"    Пример {i+1}: Предсказано={predictions_onnx[i]}, Истина={sample_labels_true[i]}")

        # Сравним с Keras моделью (если она доступна глобально - не лучший стиль, но для примера)
        # predictions_keras = np.argmax(model.predict(sample_images), axis=1)
        # print(f"  Совпадают ли предсказания Keras и ONNX: {np.array_equal(predictions_keras, predictions_onnx)}")

        print("Проверка ONNX модели завершена.")

    except Exception as e:
        print(f"Ошибка проверки ONNX модели: {e}")

In [27]:
if __name__ == "__main__":

    # --- Параметры ---
    # Задайте путь к вашим данным на Google Drive
    # Убедитесь, что ваш Google Drive смонтирован в Colab
    # Пример: from google.colab import drive; drive.mount('/content/drive')
    DATA_DIR = "./dataset/20class/SessionAllLayers"

    CLASS_NUM = 20              # Количество классов
    TRAIN_STEPS = 4000         # Общее количество шагов обучения (TRAIN_ROUND)
    BATCH_SIZE = 50             # Размер батча
    MODEL_SAVE_DIR = './trained_models_colab' # Директория для сохранения моделей в Colab
    LOAD_WEIGHTS_PATH = False #"./trained_models_colab/model_20class_SessionAllLayers_logits/model_20class_SessionAllLayers_logits.keras"    # Укажите путь к .h5 файлу, если хотите загрузить веса, например: '/content/drive/MyDrive/мои_веса.h5'
    SKIP_TRAIN = False          # Установите True, если хотите пропустить обучение (требует LOAD_WEIGHTS_PATH)
    EXPORT_ONNX = True          # Экспортировать ли модель в ONNX
    TEST_ONNX = True            # Проверять ли ONNX модель (требует EXPORT_ONNX)
    NO_SOFTMAX = True           # Создать модель БЕЗ финального Softmax (Рекомендуется для NMDL)
    # ------------------

    # Проверка аргументов (логика из парсера)
    if SKIP_TRAIN and not LOAD_WEIGHTS_PATH:
        print("Ошибка: SKIP_TRAIN установлен в True, но LOAD_WEIGHTS_PATH не указан.")
        sys.exit(1)
    if TEST_ONNX and not EXPORT_ONNX:
        print("Предупреждение: TEST_ONNX установлен в True, но EXPORT_ONNX в False. Экспорт будет выполнен.")
        EXPORT_ONNX = True

    print("--- Параметры Запуска ---")
    print(f"DATA_DIR:          {DATA_DIR}")
    print(f"CLASS_NUM:         {CLASS_NUM}")
    print(f"TRAIN_STEPS:       {TRAIN_STEPS}")
    print(f"BATCH_SIZE:        {BATCH_SIZE}")
    print(f"MODEL_SAVE_DIR:    {MODEL_SAVE_DIR}")
    print(f"LOAD_WEIGHTS_PATH: {LOAD_WEIGHTS_PATH}")
    print(f"SKIP_TRAIN:        {SKIP_TRAIN}")
    print(f"EXPORT_ONNX:       {EXPORT_ONNX}")
    print(f"TEST_ONNX:         {TEST_ONNX}")
    print(f"NO_SOFTMAX:        {NO_SOFTMAX}")
    print("-------------------------")

    # Загрузка данных
    try:
        data_loader = MNISTDataLoader(DATA_DIR, one_hot=True, class_num=CLASS_NUM)
    except FileNotFoundError as e:
        print(f"Ошибка: Директория данных или файлы не найдены! {e}")
        sys.exit(1)
    except Exception as e:
         print(f"Непредвиденная ошибка при загрузке данных: {e}")
         sys.exit(1)

    # Определяем пути для сохранения
    folder_name = os.path.basename(DATA_DIR.rstrip('/')) or f"data_{CLASS_NUM}class" # Убираем слэш в конце если есть
    model_base_name = f"model_{CLASS_NUM}class_{folder_name}"
    if NO_SOFTMAX:
        model_base_name += "_logits"
    # Убедимся, что основная директория для сохранения существует
    os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
    model_save_sub_dir = os.path.join(MODEL_SAVE_DIR, model_base_name) # Поддиректория для конкретной модели
    model_save_path = os.path.join(model_save_sub_dir, model_base_name + ".keras")
    onnx_save_path = os.path.join(model_save_sub_dir, model_base_name + ".onnx")

    # Создание модели
    model = create_original_model(CLASS_NUM, include_softmax=(not NO_SOFTMAX))

    # Обучение или загрузка весов
    if not SKIP_TRAIN:
        if LOAD_WEIGHTS_PATH:
            if os.path.exists(LOAD_WEIGHTS_PATH):
                try:
                    print(f"Загрузка весов из: {LOAD_WEIGHTS_PATH}")
                    # Загружаем только веса, предполагая совпадение архитектуры
                    model.load_weights(LOAD_WEIGHTS_PATH)
                    print("Веса загружены успешно.")
                except Exception as e:
                    print(f"Ошибка загрузки весов из {LOAD_WEIGHTS_PATH}: {e}")
                    print("Начинаем обучение с нуля.")
                    # Переходим к обучению, т.к. загрузка не удалась
                    train_model(model, data_loader, TRAIN_STEPS, BATCH_SIZE, model_save_path)
            else:
                print(f"Предупреждение: Файл весов {LOAD_WEIGHTS_PATH} не найден. Начинаем обучение с нуля.")
                train_model(model, data_loader, TRAIN_STEPS, BATCH_SIZE, model_save_path)
        else:
            # Обучаем с нуля
            train_model(model, data_loader, TRAIN_STEPS, BATCH_SIZE, model_save_path)
            print(f"Лучшая модель сохранена в: {model_save_path}")
            # Загружаем лучшую модель после обучения для последующих шагов
            print("Загрузка лучшей сохраненной модели для оценки/экспорта...")
            try:
                # Перезагружаем модель, которая была сохранена колбэком
                # Компиляция сохранится, если save_weights_only=False (по умолчанию)
                model = tf.keras.models.load_model(model_save_path)
                print("Лучшая модель загружена.")
            except Exception as e:
                print(f"Ошибка загрузки лучшей модели из {model_save_path}: {e}")
                print("Оценка и экспорт будут выполнены с моделью в текущем состоянии.")


    elif LOAD_WEIGHTS_PATH: # Если пропустили трейн, но указали веса
         if os.path.exists(LOAD_WEIGHTS_PATH):
             try:
                 print(f"Загрузка весов из: {LOAD_WEIGHTS_PATH}")
                 # Загружаем только веса в созданную архитектуру
                 model.load_weights(LOAD_WEIGHTS_PATH)
                 print("Веса загружены успешно.")
             except Exception as e:
                 print(f"Ошибка загрузки весов из {LOAD_WEIGHTS_PATH}: {e}")
                 sys.exit(1)
         else:
             print(f"Ошибка: Файл весов {LOAD_WEIGHTS_PATH} не найден, обучение пропущено.")
             sys.exit(1)
    else:
        print("Ошибка: Обучение пропущено (SKIP_TRAIN=True), но не указан путь для загрузки весов (LOAD_WEIGHTS_PATH).")
        sys.exit(1)

    # Оценка модели (уже обученной или с загруженными весами)
    if model: # Убедимся, что модель существует
        evaluate_model(model, data_loader, CLASS_NUM, DATA_DIR)
    else:
        print("Ошибка: Модель не была обучена или загружена. Пропуск оценки.")
        sys.exit(1)


    # Экспорт в ONNX
    onnx_exported = False
    if EXPORT_ONNX and model:
       os.makedirs(model_save_sub_dir, exist_ok=True) # Убедимся что директория есть
       onnx_exported = export_to_onnx(model, onnx_save_path)

    # Проверка ONNX
    if TEST_ONNX and onnx_exported:
        test_onnx_model(onnx_save_path, data_loader)
    elif TEST_ONNX and not onnx_exported:
        print("Пропуск проверки ONNX: экспорт не удался или был пропущен.")

    print("\nСкрипт завершен.")

--- Параметры Запуска ---
DATA_DIR:          ./dataset/20class/SessionAllLayers
CLASS_NUM:         20
TRAIN_STEPS:       4000
BATCH_SIZE:        50
MODEL_SAVE_DIR:    ./trained_models_colab
LOAD_WEIGHTS_PATH: False
SKIP_TRAIN:        False
EXPORT_ONNX:       True
TEST_ONNX:         True
NO_SOFTMAX:        True
-------------------------
Загрузка данных из: ./dataset/20class/SessionAllLayers
Данные загружены.
  Train: (128433, 784), (128433, 20)
  Test:  (14267, 784), (14267, 20)
--- Модель (Functional API) создана ---


/home/n1sko/tf_venv/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "SequentialOriginalLike_20class"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_3 (Reshape)             │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 28, 28, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 14, 14, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 1024)           │     3,212,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_logits (Dense)           │ (None, 20)             │        20,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,284,884 (12.53 MB)

...

 Non-trainable params: 0 (0.00 B)

-------------------------------------------


I0000 00:00:1743734386.040899  159673 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
I0000 00:00:1743734386.041079  159673 single_machine.cc:374] Starting new session
I0000 00:00:1743734386.041623  159673 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13553 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1743734386.128475  159673 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13553 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1743734386.148072  159673 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
I0000 00:00:1743734386.148244  159673 single_machine.cc:374] Starting new session
I0000 00:00:1743734386.148664  159673 gpu_device.cc:2019] Created device /job:localhost/replica:

Модель успешно экспортирована в ONNX: ./trained_models_colab/model_20class_SessionAllLayers_logits/model_20class_SessionAllLayers_logits.onnx

Проверка ONNX модели: ./trained_models_colab/model_20class_SessionAllLayers_logits/model_20class_SessionAllLayers_logits.onnx
*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/python/onnxruntime_pybind_state.cc:505 void onnxruntime::python::RegisterTensorRTPluginsAsCustomOps(PySessionOptions&, const onnxruntime::ProviderOptions&) Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
****************************************
  ONNX Input: 'input_flat', Output: 'SequentialOriginalLike_20class'
  Примеры предсказаний ONNX:
    Пример 1: Предсказано=1

2025-04-04 09:39:46.352275331 [E:onnxruntime:Default, provider_bridge_ort.cc:2022 TryGetProviderInfo_TensorRT] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1695 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_tensorrt.so with error: libnvinfer.so.10: cannot open shared object file: No such file or directory

